In [16]:
import pandas as pd
import torch
import numpy as np

In [17]:
df_Train = pd.read_csv("nb/train.csv")
ds = torch.utils.data.TensorDataset(
    torch.tensor(df_Train.loc[:, "POS_0_9_CP":"POS_90_CP"].values),
    torch.tensor(df_Train.loc[:, ["DTH_CUM_CP"]].values)
)

df_test = pd.read_csv("nb/test.csv")
ds_Test = torch.utils.data.TensorDataset(
    torch.tensor(df_test.loc[:, "POS_0_9_CP":"POS_90_CP"].values),
    torch.tensor(df_test.loc[:, ["DTH_CUM_CP"]].values)
)

In [18]:
trainX, trainY = ds[:]
testX, testY = ds_Test[:]
trainX = trainX.to(torch.float64)
trainY = trainY.to(torch.float64)
testX = testX.to(torch.float64)
testY = testY.to(torch.float64)

In [19]:
#q1
row, column = trainX.size()
bytes_Used  = row * column * 8
bytes_Used

83520

In [21]:
#q2
X_float16 = trainX.to(torch.float16)
newarray = X_float16 - trainX
difference = torch.max(newarray)
difference

tensor(0., dtype=torch.float64)

In [ ]:
#q3
is_Cuda_Available = torch.cuda.is_available()
is_Cuda_Available

In [ ]:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)
coef

In [ ]:
#q4
number_of_deaths = np.dot(testX[0], coef)
number_of_deaths.item()

In [ ]:
print(testX)

In [ ]:
#q5
numberOfPred = 0
testXRowSize = testX.size()[0]
for i in testX:
    numberOfPred += np.dot(i, coef)
predictedDeaths = numberOfPred.item()/testXRowSize
predictedDeaths

In [ ]:
#q6
def f(x):
    return pow(x,2) - 8*x + 19
    
x = torch.tensor(0.0, requires_grad = True)
y = f(x)
float(y)

In [ ]:
#q7
optimizer  = torch.optim.SGD([x], lr = 0.1)

for epoch in range(100):
    y = f(x)
    y.backward()
    optimizer.step()
    optimizer.zero_grad()
float(x)

In [ ]:
#q8
loss_fn = torch.nn.MSELoss()
batchX, batchY = ds[:]
batchX = batchX.to(torch.float64)
batchY = batchY.to(torch.float64)

coef = torch.zeros((batchX.shape[1], 1), dtype=torch.float64, requires_grad = True)
optimizer  = torch.optim.SGD([coef], lr = 0.1)
prediction = batchX @ coef
loss = loss_fn(prediction, batchY)
float(loss)

In [ ]:
#q9
torch.manual_seed(544)
ds_9 = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds_9, batch_size=50, shuffle=True)
optimizer_9  = torch.optim.SGD([coef], lr = 0.000002)

for epoch in range(500):
    for batchX, batchY in dl:
        pred_9 = batchX.to(torch.float64) @ coef
        loss_9 = loss_fn(pred_9, batchY.to(torch.float64))
        loss_9.backward()
        optimizer_9.step()
        optimizer_9.zero_grad()
    X, Y = ds[:]
    loss_9 = loss_fn(X.to(torch.float64) @ coef, Y.to(torch.float64))
float_loss = float(loss_9)
float_loss

In [ ]:
#q10
X, Y = ds_Test[:]
loss_10 = loss_fn(X.to(torch.float64) @ coef, Y.to(torch.float64))
float(loss_10)